In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

In [119]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Funciones

In [296]:
def doClean(a):   
    a = str(a)
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', a)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

In [640]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

#### Carga y union de ficheros

In [164]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [165]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'union_ficheros.ipynb',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 'union_patri.csv',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'sociedad_sanidad_centros_salud_2019_limpio.csv',
 '.ipynb_checkpoints',
 '.~lock.Empresas-SICTED2.xls#',
 'economia_turismo_establecimientos_20XX_limpio.csv',
 'tratamiento',
 'sociedad_usos_suelo_2019_limpio.csv',
 'manual.txt',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'listado-codigos-postales-con-LatyLon.csv',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv']

## Fichero maestro

In [720]:
maestro_municipios = pd.read_csv("../maestro_idiomas_ine.csv", index_col=0, dtype={'postal_code': object})
maestro_municipios.shape

(14729, 17)

In [721]:
#correcciones
maestro_municipios['province'].replace("islas_baleares", "illes_balears", inplace=True )
maestro_municipios['cities_es/province'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("comunidad_valenciana", "comunitat_valenciana", inplace=True, regex=True)
maestro_municipios['province'].replace("guipuzcoa", "gipuzkoa", inplace=True )
#maestro_municipios['cities_es/province'].replace("guipuzcoa", "gipuzkoa", inplace=True, regex=True)

#eliminacion nans
maestro_municipios = maestro_municipios[~maestro_municipios['province'].isnull()]

In [722]:
maestro_municipios.head()

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
3,1002,01468,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
4,1002,01470,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco


In [723]:
maestro_municipios.shape

(14288, 17)

### Fichero nombres originales

In [814]:
municipios = pd.read_csv("listado-codigos-postales-con-LatyLon.csv", sep=';', dtype={'codigopostalid': object})
municipios.shape

(14665, 5)

In [815]:
municipios.head(2)

,provincia,poblacion,codigopostalid,lat,lon
0,Araba/Álava,Alegría-Dulantzi,00240,-2.712437,42.939812
1,Araba/Álava,Alegría-Dulantzi,01193,-2.712437,42.939812


In [816]:
#municipios = municipios[municipios['provincia_limpio']!='albacete']


In [817]:
municipios.drop_duplicates(subset=['poblacion','provincia'], inplace=True)
municipios['provincia_limpio'] = municipios['provincia'].apply(doClean)
municipios['municipio_limpio'] = municipios['poblacion'].apply(doClean)
municipios.shape

(8098, 7)

#### matching de provincias

In [818]:
maestro_municipios_prov = pd.DataFrame(maestro_municipios['province'].drop_duplicates())
maestro_municipios_prov['province'] = maestro_municipios_prov['province'].astype(str)
maestro_municipios_prov.shape

(52, 1)

In [819]:
municipios_prov = pd.DataFrame(municipios['provincia_limpio'].drop_duplicates())
municipios_prov.shape

(52, 1)

In [850]:
def union_muni(df_maestro, df_union, muni_maestro, agrup_maestro, muni_union, agrup_union, threshold=85):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])

        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='left')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    print(df_union_agrup_match.columns)
    df_union_agrup_match.drop(columns='matches')
    
    #fuzzy por municipio
    df_maestro['muni_agrup'] = df_maestro[muni_maestro]+"/"+df_maestro[agrup_maestro]
    df_union_agrup_match['muni_agrup'] = df_union_agrup_match[muni_union]+"/"+df_union_agrup_match[agrup_maestro]
   
    muni_match = pd.DataFrame()
    maestro_count = 0
    for agrup in list(maestro_agrup[agrup_maestro]):
        print(agrup)
        maestro_agrup_sel = df_maestro[df_maestro[agrup_maestro] == agrup].copy()
        maestro_count = maestro_agrup_sel.shape[0] + maestro_count
        union_agrup_sel = df_union_agrup_match[df_union_agrup_match[agrup_maestro] == agrup].copy()
        aux = fuzzy_merge(maestro_agrup_sel, union_agrup_sel, 'muni_agrup', 'muni_agrup', threshold=threshold)
        num_matches = (aux[aux['matches']!='']['matches'].nunique())
        if (num_matches != aux[muni_maestro].nunique()):
            print('no se han cruzado todos los valores:')
            #print(aux[aux['matches'].isnull()]) 
        muni_match = pd.concat([muni_match, aux])
        
        #comprobacion registros    
    print('num muni maestro: {}'.format(maestro_count))
    if maestro_count != muni_match.shape[0]:
        print("se han perdido registros")       
    
    print(muni_match.columns)
        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(muni_match, df_union, left_on='matches', right_on=muni_union, how='left')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar por municipio')        

    #merge maestro con fichero union
    #df_tras_union = pd.merge(df_maestro, df_union_muni_match, left_on=muni_maestro, right_on='matches', how='left')
    
    print('columns final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    if muni_maestro != muni_union:
        df_tras_union.drop(columns=muni_union, inplace=True)
    if agrup_maestro != agrup_union:
        df_tras_union.drop(columns=agrup_union, inplace=True)
    df_tras_union.drop(columns='matches')
    print('num registros tras union: {}'.format(df_tras_union.shape))
    
    return df_tras_union

In [851]:
a = union_muni(maestro_municipios, municipios, 'cities', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85)
a.head()

num registros maestro: 14288
num registros fichero para unir: 8098
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
Index(['provincia', 'poblacion', 'codigopostalid', 'lat', 'lon',
       'provincia_limpio', 'municipio_limpio', 'province', 'matches'],
      dtype='object')
araba
no se han cruzado todos los valores:
albacete
alicante
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
avila
badajoz
illes_balears
no se han cruzado todos los valores:
barcelona
no se han cruzado todos los valores:
tarragona
no se han cruzado todos los valores:
burgos
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
palencia
no se han cruzado todos los valores:
caceres
cadiz
castellon
no se han cruzado todos los valores:
ciudad_real
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
coruna
cuenca
leo

NameError: name 'df_tras_union' is not defined

In [740]:
provincia_match = fuzzy_merge(maestro_municipios_prov, municipios_prov, 'province', 'provincia_limpio', threshold=85)
provincia_match.head(60)

,province,matches
0,araba,araba
132,albacete,
305,alicante,alicante
496,murcia,murcia
549,almeria,almeria
768,avila,avila
1063,badajoz,badajoz
1283,illes_balears,illes_balears
1463,barcelona,barcelona
1536,tarragona,tarragona


In [704]:
municipios_merged = pd.merge(municipios, provincia_match, left_on='provincia_limpio', right_on='matches', how='left')
municipios_merged.shape

(14665, 9)

In [705]:
municipios_merged.head(2)

,provincia,poblacion,codigopostalid,lat,lon,provincia_limpio,municipio_limpio,province,matches
0,Araba/Álava,Alegría-Dulantzi,240,-2.712437,42.939812,araba,alegria_dulantzi,araba,araba
1,Araba/Álava,Alegría-Dulantzi,1193,-2.712437,42.939812,araba,alegria_dulantzi,araba,araba


#### matching de municipios

In [706]:
municipios_merged['cities_es/province'] = municipios_merged['municipio_limpio']+"/"+municipios_merged['province']
municipios_merged.head(2)

,provincia,poblacion,codigopostalid,lat,lon,provincia_limpio,municipio_limpio,province,matches,cities_es/province
0,Araba/Álava,Alegría-Dulantzi,240,-2.712437,42.939812,araba,alegria_dulantzi,araba,araba,alegria_dulantzi/araba
1,Araba/Álava,Alegría-Dulantzi,1193,-2.712437,42.939812,araba,alegria_dulantzi,araba,araba,alegria_dulantzi/araba


In [709]:
maestro = pd.DataFrame()
a_count = 0
b_count = 0
aux_count = 0
maestro_count = 0
for prov in list(maestro_municipios_prov['province']):
    print(prov)
    a = maestro_municipios[maestro_municipios['province'] == prov].copy()
    a_count = a.shape[0]+a_count
    print('a count= {}'.format(a_count))
    b = municipios_merged[municipios_merged['province'] == prov].copy()
    b_count = b.shape[0]+b_count
    print('b count= {}'.format(b_count))
    aux = fuzzy_merge(a, b, 'cities_es/province', 'cities_es/province', threshold=90)
    aux_count = aux.shape[0]+aux_count
    print('aux_count count= {}'.format(aux_count))
    maestro = pd.concat([maestro, aux])
    maestro_count = maestro.shape[0]
    print('maestro_count= {}'.format(maestro_count))

araba
a count= 136
b count= 132
aux_count count= 136
maestro_count= 136
albacete
a count= 288
b count= 305
aux_count count= 288
maestro_count= 288
alicante
a count= 531
b count= 549
aux_count count= 531
maestro_count= 531
murcia
a count= 756
b count= 788
aux_count count= 756
maestro_count= 756
almeria
a count= 970
b count= 1007
aux_count count= 970
maestro_count= 970
avila
a count= 1251
b count= 1302
aux_count count= 1251
maestro_count= 1251
badajoz
a count= 1470
b count= 1521
aux_count count= 1470
maestro_count= 1470
illes_balears
a count= 1648
b count= 1701
aux_count count= 1648
maestro_count= 1648
barcelona
a count= 2092
b count= 2149
aux_count count= 2092
maestro_count= 2092
tarragona
a count= 2347
b count= 2407
aux_count count= 2347
maestro_count= 2347
burgos
a count= 2883
b count= 2949
aux_count count= 2883
maestro_count= 2883
cantabria
a count= 3150
b count= 3240
aux_count count= 3150
maestro_count= 3150
la_rioja
a count= 3356
b count= 3445
aux_count count= 3356
maestro_count= 3

In [710]:
maestro.shape

(14288, 18)

In [843]:
maestro.head(60)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,matches
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,alegria_dulantzi/araba
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,alegria_dulantzi/araba
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,amurrio/araba
3,1002,01468,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,amurrio/araba
4,1002,01470,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,amurrio/pais_vasco,amurrio/pais_vasco,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,amurrio/araba
5,1003,01160,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,aramaio/pais_vasco,aramaio/pais_vasco,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,aramaio/araba
6,1003,01165,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,aramaio/pais_vasco,aramaio/pais_vasco,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,aramaio/araba
7,1003,01169,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,aramaio/pais_vasco,aramaio/pais_vasco,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,aramaio/araba
8,1004,01474,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/alava,artziniega/araba,artziniega/pais_vasco,artziniega/pais_vasco,artziniega/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,artziniega/araba
9,1004,01478,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/alava,artziniega/araba,artziniega/pais_vasco,artziniega/pais_vasco,artziniega/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,artziniega/araba


In [711]:
maestro_municipios.head()

(14288, 17)

In [715]:
maestro_merged = pd.merge(maestro, municipios_merged, left_on='matches', right_on='cities_es/province', how='left')
maestro_merged.shape

(70863, 28)

In [19]:
maestro_municipios[maestro_municipios['cities_es']=='benicasim']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
2905,12028,12560,benicasim,benicassim,castellon,castello,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castello,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana
2919,12040,12560,benicasim,benicassim,castellon,castello,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castello,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana


In [20]:
maestro_municipios[maestro_municipios['province_es']=='castellon'].head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
2875,12001,12132,atzeneta_del_maestrat,atzeneta_del_maestrat,castellon,castello,comunidad_valenciana,comunitat_valenciana,atzeneta_del_maestrat/castellon,atzeneta_del_maestrat/castello,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana
2876,12001,12133,atzeneta_del_maestrat,atzeneta_del_maestrat,castellon,castello,comunidad_valenciana,comunitat_valenciana,atzeneta_del_maestrat/castellon,atzeneta_del_maestrat/castello,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/comunidad_valenciana,atzeneta_del_maestrat/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana


## Ficheros especificos

##### Bibliotecas

In [21]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017_limipio.csv", index_col=0)

In [22]:
bibliotecas.shape

(3323, 4)

In [23]:
bibliotecas.head(2)

,municipio_limpio,ccaa,provincia,total_bibliotecas
0,alaior,illes_balears,illes_balears,1
1,alaro,illes_balears,illes_balears,1


In [24]:
bibliotecas[bibliotecas['municipio_limpio']=='segorbe']

,municipio_limpio,ccaa,provincia,total_bibliotecas
3313,segorbe,comunidad_valenciana,castellon_castello,2


In [25]:
bibliotecas[bibliotecas['municipio_limpio']=='palma']
# idioma municipio = es

,municipio_limpio,ccaa,provincia,total_bibliotecas
38,palma,illes_balears,illes_balears,22


In [26]:
maestro_municipios[maestro_municipios['cities']=='palma']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
1299,7010,07010,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1343,7031,07600,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1365,7040,07600,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1366,7040,07012,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1368,7040,07199,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1369,7040,07198,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1370,7040,07120,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1371,7040,07071,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1372,7040,07015,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears
1373,7040,07014,palma,palma,islas_baleares,illes_balears,islas_baleares,illes_balears,palma/islas_baleares,palma/illes_balears,palma/islas_baleares,palma/illes_balears,palma/illes_balears,illes_balears/islas_baleares,illes_balears/illes_balears,islas_baleares/islas_baleares,islas_baleares/illes_balears


In [27]:
bibliotecas['ccaa'].replace("illes_balears","islas_baleares", inplace=True, regex=True)
bibliotecas['ccaa'].replace("comunidad_valenciana","comunitat_valenciana", inplace=True, regex=True)

In [28]:
bibliotecas['cities_es/ccaa_es'] = bibliotecas['municipio_limpio']+"/"+bibliotecas['ccaa']

In [29]:
bibliotecas = bibliotecas.drop(columns=['municipio_limpio', 'ccaa', 'provincia'])

In [30]:
union = pd.merge(maestro_municipios, bibliotecas, left_on='cities_es/ccaa_es', right_on='cities_es/ccaa_es', how='left')

In [31]:
union[~union['total_bibliotecas'].isnull()].shape

(7350, 18)

In [32]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0


In [33]:
union.shape

(14729, 18)

##### Hospitales

In [34]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [35]:
hospitales.shape

(613, 2)

In [36]:
hospitales.head(2)

,cp,uds_hospiatales
0,01002,1
1,01006,1


In [37]:
hospitales[hospitales['cp']=='46010']

,cp,uds_hospiatales
549,46010,2


In [38]:
union = pd.merge(union, hospitales, left_on='postal_code', right_on='cp', how='left')

In [39]:
union[~union['uds_hospiatales'].isnull()].shape

(650, 20)

In [40]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,cp,uds_hospiatales
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN


In [41]:
union = union.drop(columns=['cp'])

In [42]:
union.shape

(14729, 19)

##### Centros salud

In [43]:
centros_salud = pd.read_csv("sociedad_sanidad_centros_salud_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [44]:
centros_salud.shape

(8316, 2)

In [45]:
centros_salud.head(2)

,cp,num_centros_salud
0,01001,1
1,01002,2


In [46]:
centros_salud[centros_salud['cp']=='46010']

,cp,num_centros_salud
7249,46010,1


In [47]:
union = pd.merge(union, centros_salud, left_on='postal_code', right_on='cp', how='left')

In [48]:
union[~union['num_centros_salud'].isnull()].shape

(11332, 21)

In [49]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,uds_hospiatales,cp,num_centros_salud
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,01240,1.0
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,NaN


In [50]:
union = union.drop(columns=['cp'])

In [51]:
union.shape

(14729, 20)

##### Colegios

In [52]:
colegios = pd.read_csv("sociedad_educacion_colegios_2020_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [53]:
colegios.shape

(7838, 5)

In [54]:
colegios.head(2)

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
0,00000,a_coruna,a_coruna,6.0,NaN
1,00000,a_pontenova,lugo,NaN,1.0


In [55]:
colegios[colegios['codigo_postal']=='01450']

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos


In [56]:
#codigos postales erroneos, se une por municipio/provincia y cambiando de idioma

In [57]:
colegios[colegios['municipio_limpio']=='alegria_dulantzi']
# idioma municipio = es

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
81,01240,alegria_dulantzi,araba,NaN,3.0


In [58]:
maestro_municipios[maestro_municipios['cities']=='alegria_dulantzi']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
42,1021,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
45,1027,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
106,1059,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco


In [59]:
maestro_municipios['province'].drop_duplicates().sort_values()

305                     alacant
132                    albacete
549                     almeria
0                         araba
9209                   asturias
768                       avila
1063                    badajoz
1463                  barcelona
13776                   bizkaia
1911                     burgos
2455                    caceres
2759                      cadiz
1918                  cantabria
2875                   castello
14718                     ceuta
3044                ciudad_real
3070                    cordoba
3379                     coruna
3801                     cuenca
4087                     girona
3311                    granada
4663                guadalajara
5051                  guipuzcoa
5183                     huelva
5214                     huesca
1283              illes_balears
5738                       jaen
2170                   la_rioja
9962                 las_palmas
3985                       leon
6509                     lleida
7124    

In [60]:
colegios['provincia'].replace("alicante", "alacant", inplace=True )
colegios['provincia'].replace("castellon", "castello", inplace=True )
colegios['provincia'].replace("a_coruna", "coruna", inplace=True, regex=True)

In [61]:
# listado de provincias y ccaa
colegios['provincia'].drop_duplicates().sort_values()

188                    alacant
103                   albacete
378                    almeria
45                       araba
4997                  asturias
14                       avila
577                    badajoz
966                  barcelona
7451                   bizkaia
4                       burgos
9                      caceres
1590                     cadiz
6213                 cantabria
1711                  castello
7827                     ceuta
12                 ciudad_real
1893                   cordoba
0                       coruna
2470                    cuenca
2992                  gipuzkoa
2528                    girona
2708                   granada
18                 guadalajara
3130                    huelva
3228                    huesca
773              illes_balears
3316                      jaen
3779                  la_rioja
5259                las_palmas
2379                      leon
3580                    lleida
1                         lugo
3957    

In [62]:
colegios['cities_es/province'] = colegios['municipio_limpio']+"/"+colegios['provincia']

In [63]:
colegios = colegios.groupby(by='cities_es/province', as_index=False).agg({'num_colegios_privados':'sum', 'num_colegios_publicos':'sum'})

In [64]:
union = pd.merge(union, colegios, left_on='cities_es/province', right_on='cities_es/province', how='left')

In [65]:
union[~union['num_colegios_publicos'].isnull()].shape

(7874, 22)

In [66]:
union.head(10)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,uds_hospiatales,num_centros_salud,num_colegios_privados,num_colegios_publicos
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,0.0,3.0
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,0.0,3.0
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,...,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,3.0,6.0
3,1002,01468,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,...,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,3.0,6.0
4,1002,01470,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,...,amurrio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,3.0,6.0
5,1003,01160,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,...,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,NaN,NaN
6,1003,01165,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,...,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,NaN,NaN
7,1003,01169,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/alava,aramaio/araba,...,aramaio/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,NaN,NaN
8,1004,01474,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/alava,artziniega/araba,...,artziniega/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,1.0,2.0
9,1004,01478,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/alava,artziniega/araba,...,artziniega/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,1.0,2.0


In [67]:
union.columns

Index(['ine_code', 'postal_code', 'cities_es', 'cities', 'province_es',
       'province', 'ccaa_es', 'ccaa', 'cities_es/province_es',
       'cities_es/province', 'cities_es/ccaa_es', 'cities_es/ccaa',
       'cities/province_es', 'province_es/ccaa_es', 'province_es/ccaa',
       'province/ccaa_es', 'province/ccaa', 'total_bibliotecas',
       'uds_hospiatales', 'num_centros_salud', 'num_colegios_privados',
       'num_colegios_publicos'],
      dtype='object')

In [68]:
union.shape

(14729, 22)

##### Guardia Civil

In [69]:
guardia_civil = pd.read_csv("sociedad_seguridad_guardia_civil_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [70]:
guardia_civil.shape

(2237, 4)

In [71]:
guardia_civil.head(2)
#codigo postal no valido

,cp,municipio_limpio,provincia,uds_guardia_civil
0,10003,caceres,caceres,1
1,1010,vitoria,alava,1


In [72]:
guardia_civil[guardia_civil['municipio_limpio']=='benicasim']
# idioma municipio = es

,cp,municipio_limpio,provincia,uds_guardia_civil
153,12560,benicasim,castellon,1


In [73]:
maestro_municipios[maestro_municipios['cities_es']=='benicasim']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
2905,12028,12560,benicasim,benicassim,castellon,castello,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castello,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana
2919,12040,12560,benicasim,benicassim,castellon,castello,comunidad_valenciana,comunitat_valenciana,benicasim/castellon,benicasim/castello,benicasim/comunidad_valenciana,benicasim/comunidad_valenciana,benicassim/castellon,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana,castellon/comunidad_valenciana


In [74]:
maestro_municipios['province_es'].drop_duplicates().sort_values()

0                         alava
132                    albacete
305                    alicante
549                     almeria
9209                   asturias
768                       avila
1063                    badajoz
1463                  barcelona
1911                     burgos
2455                    caceres
2759                      cadiz
1918                  cantabria
2875                  castellon
14718                     ceuta
3044                ciudad_real
3070                    cordoba
3379                     coruna
3801                     cuenca
4087                     girona
3311                    granada
4663                guadalajara
5051                  guipuzkoa
5183                     huelva
5214                     huesca
1283             islas_baleares
5738                       jaen
2170                   la_rioja
9962                 las_palmas
3985                       leon
6509                     lleida
7124                       lugo
4739    

In [75]:
guardia_civil['provincia'].replace("illes_balears", "islas_baleares", inplace=True )

In [76]:
# listado de provincias y ccaa
guardia_civil['provincia'].drop_duplicates().sort_values()

284                   a_coruna
1                        alava
510                   albacete
1032                  alicante
1492                   almeria
1182                  asturias
1944                     avila
2024                   badajoz
2166                 barcelona
2200                    burgos
0                      caceres
75                       cadiz
1449                 cantabria
128                  castellon
1994                     ceuta
164                ciudad_real
231                    cordoba
341                     cuenca
509                   gipuzkoa
388                     girona
400                    granada
477                guadalajara
520                     huelva
585                     huesca
2117            islas_baleares
639                       jaen
777                   la_rioja
1295                las_palmas
711                       leon
759                     lleida
819                       lugo
871                     madrid
973     

In [77]:
guardia_civil['cities_es/province_es'] = guardia_civil['municipio_limpio']+"/"+guardia_civil['provincia']

In [78]:
guardia_civil = guardia_civil.groupby(by='cities_es/province_es', as_index=False).agg({'uds_guardia_civil':'sum'})

In [79]:
union = pd.merge(union, guardia_civil, left_on='cities_es/province_es', right_on='cities_es/province_es', how='left')

In [80]:
union[~union['uds_guardia_civil'].isnull()].shape

(4763, 23)

In [81]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,total_bibliotecas,uds_hospiatales,num_centros_salud,num_colegios_privados,num_colegios_publicos,uds_guardia_civil
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,1.0,0.0,3.0,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco,1.0,NaN,NaN,0.0,3.0,NaN


In [82]:
union.shape

(14729, 23)

##### Universidades

In [83]:
universidad = pd.read_csv('sociedad_educacion_universidad_2018_limpio.csv', index_col=0, dtype={'cp': object})

In [84]:
universidad.shape

(165, 6)

In [85]:
universidad.head(2)

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0
1,alcala_de_henares,madrid,1.0,1.0,5.0,114.0


In [86]:
universidad[universidad['municipio_limpio']=='alcoy']

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
5,alcoy,comunitat_valenciana,NaN,1.0,NaN,12.0


In [87]:
maestro_municipios[maestro_municipios['cities']=='alcoy']

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
315,3009,03804,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
316,3009,03803,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
317,3009,03802,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
318,3009,03800,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana
319,3009,03801,alcoi,alcoy,alicante,alacant,comunidad_valenciana,comunitat_valenciana,alcoi/alicante,alcoi/alacant,alcoi/comunidad_valenciana,alcoi/comunidad_valenciana,alcoy/alicante,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana,alicante/comunidad_valenciana


In [88]:
universidad['ccaa'].drop_duplicates().sort_values()

7                andalucia
11                  aragon
60                asturias
50                 balears
18                canarias
42               cantabria
0       castilla_la_mancha
19         castilla_y_leon
21                cataluna
2     comunitat_valenciana
9              extremadura
45                 galicia
1                   madrid
32                  murcia
22                 navarra
16              pais_vasco
78                   rioja
Name: ccaa, dtype: object

In [89]:
universidad['ccaa'].replace("balears","illes_balears", inplace=True, regex=True)
universidad['ccaa'].replace("madrid","comunidad_de_madrid", inplace=True, regex=True)
universidad['ccaa'].replace("navarra","comunidad_foral_de_navarra", inplace=True, regex=True)
universidad['ccaa'].replace("asturias","principado_de_asturias", inplace=True, regex=True)
universidad['ccaa'].replace("rioja","la_rioja", inplace=True, regex=True)
universidad['ccaa'].replace("murcia","region_de_murcia", inplace=True, regex=True)

In [90]:
maestro_municipios['ccaa'].drop_duplicates().sort_values()

549                       andalucia
5214                         aragon
9962                       canarias
1918                      cantabria
132              castilla_la_mancha
768                 castilla_y_leon
1463                       cataluna
14718                         ceuta
4739            comunidad_de_madrid
8408     comunidad_foral_de_navarra
305            comunitat_valenciana
1063                    extremadura
3379                        galicia
1283                  illes_balears
2170                       la_rioja
14723                       melilla
0                        pais_vasco
9209         principado_de_asturias
496                region_de_murcia
134                             NaN
Name: ccaa, dtype: object

In [91]:
universidad['cities_es/ccaa'] = universidad['municipio_limpio']+"/"+universidad['ccaa']

In [92]:
universidad.head()

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0,albacete/castilla_la_mancha
1,alcala_de_henares,comunidad_de_madrid,1.0,1.0,5.0,114.0,alcala_de_henares/comunidad_de_madrid
2,alcobendas,comunitat_valenciana,1.0,NaN,6.0,NaN,alcobendas/comunitat_valenciana
3,alcobendas,comunidad_de_madrid,3.0,NaN,26.0,NaN,alcobendas/comunidad_de_madrid
4,alcorcon,comunidad_de_madrid,NaN,1.0,NaN,21.0,alcorcon/comunidad_de_madrid


In [93]:
universidad.drop(columns= ['municipio_limpio','ccaa'], inplace=True)

In [94]:
union = pd.merge(union, universidad, left_on='cities_es/ccaa_es', right_on='cities_es/ccaa', how='left')

In [95]:
union[~union['num_universidades_publicas'].isnull()].shape

(1204, 28)

In [96]:
union.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,uds_hospiatales,num_centros_salud,num_colegios_privados,num_colegios_publicos,uds_guardia_civil,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa_y
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,1.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,NaN,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
union.shape

(14729, 28)

##### Establecimientos

In [98]:
establecimientos = pd.read_csv("economia_turismo_establecimientos_20XX_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [99]:
establecimientos.shape

(2063, 35)

In [100]:
establecimientos.head(2)

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,00000,aldea_de_cuenca_hinojares,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [101]:
# mas de un codigo postal por municipio, se producen duplicados
establecimientos[establecimientos['codigo_postal']=='01118']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
18,01118,bernedo,euskadi_pais_vasco,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,01118,lagran,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,01118,urturi,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
establecimientos[establecimientos['municipio_limpio']=='alcoy']
# municipio en 2 idiomas, se cruza por cp

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
164,03800,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
166,03801,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
168,03802,alcoy,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
169,03803,alcoy,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
171,03804,alcoy,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [103]:
establecimientos[establecimientos['codigo_postal'].duplicated()]

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,00000,cazorla,andalucia,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000,la_iruela,andalucia,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000,pozo_alcon,andalucia,NaN,1.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,01117,san_vicente_arana,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,50513,los_fayos,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2050,50513,santa_cruz_de_moncayo,aragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051,50513,santa_cruz_del_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,50513,torrellas,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# listado de provincias y ccaa
establecimientos[establecimientos['codigo_postal']=='50513']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
2047,50513,anon_de_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2048,50513,grisel,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2049,50513,los_fayos,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2050,50513,santa_cruz_de_moncayo,aragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051,50513,santa_cruz_del_moncayo,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,50513,torrellas,aragon,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
establecimientos.drop(columns=['municipio_limpio','ccaa'], inplace=True)

In [106]:
establecimientos = establecimientos.groupby(['codigo_postal'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else x.head(1))

In [107]:
union = pd.merge(union, establecimientos, left_on='postal_code', right_on='codigo_postal', how='left')

In [108]:
union[~union['agencias_de_viajes'].isnull()].shape

(1826, 61)

In [109]:
union.columns

Index(['ine_code', 'postal_code', 'cities_es', 'cities', 'province_es',
       'province', 'ccaa_es', 'ccaa', 'cities_es/province_es',
       'cities_es/province', 'cities_es/ccaa_es', 'cities_es/ccaa_x',
       'cities/province_es', 'province_es/ccaa_es', 'province_es/ccaa',
       'province/ccaa_es', 'province/ccaa', 'total_bibliotecas',
       'uds_hospiatales', 'num_centros_salud', 'num_colegios_privados',
       'num_colegios_publicos', 'uds_guardia_civil',
       'num_universidades_privadas', 'num_universidades_publicas',
       'num_tit_uni_privadas', 'num_tit_uni_publicas', 'cities_es/ccaa_y',
       'codigo_postal', 'agencias_de_viajes', 'albergues',
       'alojamientos_rurales', 'artesanos', 'atraque_de_cruceros_y_ferries',
       'bares_y_cafeterías', 'bodegas', 'campings',
       'campos_de_golf_y_campos_de_pitch&putt', 'comercios',
       'convention_bureaux',
       'empresas_de_alquiler_de_vehículos_y_embarcaciones',
       'empresas_de_transporte_turístico', 'empresas_

In [110]:
union.shape

(14729, 61)

##### Usos suelo

In [111]:
usos_suelo = pd.read_csv("sociedad_usos_suelo_2019_limpio.csv", index_col=0)

In [112]:
usos_suelo.shape

(7610, 12)

In [113]:
usos_suelo.head(2)

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,abengibre,albacete,"302,079",13,3,2,1,169,1,2,1,0
1,alatoz,albacete,"339,890",41,2,1,1,140,4,2,2,0


In [114]:
usos_suelo['provincia'].drop_duplicates()
# no esta el pais vasco - se deescarta

0                     albacete
87                    alicante
228                    almeria
331                      avila
579                    badajoz
744                    balears
811                  barcelona
1122                    burgos
1493                   caceres
1716                     cadiz
1761                 castellon
1896               ciudad_real
1998                   cordoba
2075                    coruna
2169                    cuenca
2407                    girona
2628                   granada
2802               guadalajara
3090                    huelva
3170                    huesca
3372                      jaen
3469                      leon
3680                    lleida
3911                     rioja
4085                      lugo
4152                    madrid
4331                    malaga
4434                    murcia
4479                   ourense
4571                  asturias
4649                  palencia
4840                    palmas
4874    

In [115]:
union.shape

(14729, 61)

In [116]:
union.head(3)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,1001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1002,01450,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/alava,amurrio/araba,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
union.to_csv("union_patri.csv")